# Youtube Data APIを用いたバーチャルYoutuberのデータ収集
by konkon

## 前準備
Youtube Data APIのAPI_KEYは以下の方法で参考に取得してください。

https://www.gwtcenter.com/getting-google-api-key  
https://www.koredaism.com/web-service/%E3%80%90%E5%8B%95%E7%94%BB%E3%81%BE%E3%81%A8%E3%82%81%E3%82%B5%E3%82%A4%E3%83%88%E3%80%912019%E7%89%88-youtube-api-%E8%A9%B3%E3%81%97%E3%81%84%E7%B0%A1%E5%8D%98%E3%81%AA%E5%8F%96%E5%BE%97%E6%96%B9/  

API_KEYの取得にはGoogleアカウントの作成とGCP(Google Cloud Platform)のプロジェクト作成が必要になります。
GCPプロジェクトの作成方法は以下を参照してください。

https://www.gwtcenter.com/creating-a-project-in-gcp

## 実行環境

Python >= 3.6.5  
numpy >= 1.14.5  
google-api-python-client >= 1.7.11  
tqdm >= 4.23.4  
ipywidgets >= 7.2.1  
widgetsnbextension >= 3.2.1
 
Jupyterの導入方法は以下の記事を参考にしてください。
https://qiita.com/KI1208/items/a7765e6fdc95c3e03609

### tqdmのJupyter-notebookでの使用方法
コマンドプロンプトで以下のように入力してください。
```
pip install tqdm  
pip install widgetsnbextension  
jupyter nbextension enable --py --sys-prefix widgetsnbextension  
```

## インポート

In [52]:
import re
import numpy as np
import datetime
import time

from apiclient.discovery import build

from tqdm import tqdm_notebook as tqdm
from ipywidgets import IntProgress

## APIkeyの入力

In [53]:
API_KEY='XXXXX'

## チャンネルIDの入力
チャンネルIDは、対象のYoutubeチャンネルURLから確認できます。  
例：Siro Channel(電脳少女シロ)の場合、チャンネルURLは https://www.youtube.com/channel/UCLhUvJ_wO9hOvv_yYENu4fQ であり、  
チャンネルIDはこの末尾の'UCLhUvJ_wO9hOvv_yYENu4fQ'となります。

In [65]:
id_ = 'UCUZ5AlC3rTlM-rA2cj5RP6w' #例: 神楽すず

## チャンネル情報の取得
チャンネル内の全ての動画のプレイリストIDを取得します。

In [66]:
def YoutubeChannelDetails(id_, API_KEY):
    API_SERVICE_NAME = "youtube"
    API_VERSION = "v3"

    youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)
    search_response = youtube.channels().list(
    part= 'snippet,contentDetails',
    id=id_,
    ).execute()
    
    return search_response['items'][0]

In [67]:
ChannelDetails = YoutubeChannelDetails(id_,API_KEY)

In [68]:
ChannelTitle = '-'.join(ChannelDetails['snippet']['title'].split(' '))
uploads = ChannelDetails['contentDetails']['relatedPlaylists']['uploads']

## チャンネル内の動画IDの取得

In [69]:
def YoutubePlaylistContents(id_, API_KEY):
    
    responses = []
    nextPageToken = 'start'
    counts = 0

    while(nextPageToken is not None):
        
        API_SERVICE_NAME = "youtube"
        API_VERSION = "v3"

        youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

        if(nextPageToken == 'start'):
            search_response = youtube.playlistItems().list(
            part= 'snippet',
            playlistId=id_,
            maxResults = 50,
            ).execute()
            nextPageToken = search_response['nextPageToken']
        else:
            search_response = youtube.playlistItems().list(
            part= 'snippet',
            playlistId=id_,
            maxResults = 50,
            pageToken = nextPageToken
            ).execute()
            try:
                nextPageToken = search_response['nextPageToken']
            except:
                nextPageToken = None
        
        responses.extend(search_response['items'])
        counts += len(search_response['items'])
    
    print('load '+str(counts)+' videos...')
    
    return responses

In [70]:
total_contents = YoutubePlaylistContents(uploads,API_KEY)

load 245 videos...


## 取得データからの情報抽出
チャンネル内の全ての動画のタイトル、投稿日時、動画IDを取得します。

In [71]:
dic_total = []
for t in total_contents:
    
    date_list = t['snippet']['publishedAt'].split('T')
    year, month, date = date_list[0].split('-')
    hour, minute, sec = date_list[1].split(':')
    sec = sec[:2]
    
    dic = {'title':t['snippet']['title'], 
           'date':datetime.datetime(int(year),int(month),int(date),int(hour),int(minute),int(sec)),
           'Id':t['snippet']['resourceId']['videoId']}
    
    dic_total.append(dic)

## 各動画の情報を取得
全ての動画の再生回数、高/低評価数、コメント数、再生時間を取得します。

In [72]:
def ConvertDuration(string):
    string = string.replace('PT', '') 
    strings = re.split('\D',string)[:-1]
    if(len(strings) == 3):
        delta = datetime.timedelta(hours=int(strings[0]),
                                   minutes=int(strings[1]),
                                   seconds=int(strings[2]))
    elif(len(strings) == 2):
        delta = datetime.timedelta(minutes=int(strings[0]),
                                   seconds=int(strings[1]))
    elif(len(strings) == 1):
        delta =datetime.timedelta(seconds=int(strings[0]))
    else:
        delta = datetime.timedelta(seconds=0)
    
    return delta.seconds

def YoutubeVideoDetails(id_, API_KEY):
    API_SERVICE_NAME = "youtube"
    API_VERSION = "v3"

    youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

    search_response = youtube.videos().list(
    part= 'statistics,contentDetails',
    id=id_,
    ).execute()
    
    hoge = search_response['items'][0]
    details = {'viewCount':int(hoge['statistics']['viewCount']),
               'likeCount':int(hoge['statistics']['likeCount']),
               'dislikeCount':int(hoge['statistics']['dislikeCount']),
               'commentCount':int(hoge['statistics']['commentCount']),
               'duration':ConvertDuration(str(hoge['contentDetails']['duration']))}
    
    return details

In [73]:
for n,d in enumerate(tqdm(np.array([i['Id'] for i in dic_total]))):
    details = YoutubeVideoDetails(d,API_KEY)
    dic_total[n].update(details)
dic_total = np.array(dic_total)

## 取得情報のサンプルを表示

date: 投稿日時 (datetime)  
Id: 動画ID  
viewCount: 再生数  
likeCount: 高評価数  
dislikeCount: 低評価数  
commentCount: コメント数  
duration: 再生時間(秒)  

In [96]:
dic_total[100]

{'title': 'APE OUT やる 02',
 'date': datetime.datetime(2019, 3, 29, 12, 10, 6),
 'Id': 'ijauZ-6ZdJ4',
 'viewCount': 30327,
 'likeCount': 2910,
 'dislikeCount': 7,
 'commentCount': 29,
 'duration': 3550}

## 取得したデータの保存

In [46]:
np.save(ChannelTitle+'-data.npy',dic_total)